In [ ]:
import pandas as pd
import os
import re
records = pd.read_excel(r"Result/result1.xlsx", header=[0], index_col = [0])
records

保存smt的变量、限制等信息

In [ ]:
Secondary_file = ['20170427-VeryMax', '2019-ezsmt', '20210219-Dartagnan']
for sf in Secondary_file:
    path = "../QF_NIA/"+sf
    folder_list = os.listdir(path)
    for filefolder in folder_list:
        folder_path = path + "/" + filefolder
        file_list = os.listdir(folder_path)
        total = len(file_list)
        count = 1
        for filename in file_list:
            with open(os.path.join(folder_path, filename)) as f:
                vars = 0
                assertions = 0
                line = f.readline()
                while line:
                    if line[:3] == '(de':
                        vars += 1
                    elif line[:3] == '(as':
                        assertions += 1
                    line = f.readline()
                print(count, "/", total,": ", filename, "+", vars, "+", assertions)
                count += 1
                records = records.append({'filename': sf+'/' + filefolder + '/' + filename, '#vars':vars, '#assertions':assertions}, ignore_index=True, verify_integrity=True)

path = "../QF_NIA"
folder_list = os.listdir(path)
folder_list.remove('2019-ezsmt')
folder_list.remove('20170427-VeryMax')
folder_list.remove('20210219-Dartagnan')
for filefolder in folder_list:
    folder_path = path + "/" + filefolder
    file_list = os.listdir(folder_path)
    total = len(file_list)
    count = 1
    for filename in file_list:
        with open(os.path.join(folder_path, filename)) as f:
            vars = 0
            assertions = 0
            line = f.readline()
            while line:
                if line[:3] == '(de':
                    vars += 1
                elif line[:3] == '(as':
                    assertions += 1
                line = f.readline()
            print(filefolder, "(",count, "/", total,"): ", filename, "+", vars, "+", assertions)
            count += 1
            records = records.append({'filename': filefolder + '/' + filename, '#vars':vars, '#assertions':assertions}, ignore_index=True, verify_integrity=True)
print("Waiting...")
records.to_excel('Result/records.xlsx')
print("Done")

修改filename未测试路径, 保存为z3的索引

In [ ]:
folder_path = "Data_File/z3/10"
file_list = os.listdir(folder_path)
total = len(file_list)
count = 1
for filename in file_list:
    with open(os.path.join(folder_path, filename)) as f:
        while f.readline():
            result = f.readline()
            info = f.readline().split(' : ')
            records.loc[records['filename'] == info[0].split('//')[-1], 'Problem(file name)'] = info[0]
    print("(",count, "/", total,"): ", filename)
    count += 1
print("Waiting...")
records.to_excel('Result/result1.xlsx')
print("Done")


保存求解器得到的result、timeout等信息

In [ ]:
path = "Data_File"
folder_list = os.listdir(path)
for filefolder in folder_list:
    if filefolder != 'smtrat' and filefolder != 'ismt':
        folder_path = path + "/" + filefolder
        times_list = os.listdir(folder_path)
        for filetimes in times_list:
            file_path = folder_path + "/" + filetimes
            file_list = os.listdir(file_path)
            result_index = filefolder + "_" + filetimes + "_Result"
            timeout_index = filefolder + "_" + filetimes + "_Timeout"
            total = len(file_list)
            count = 1
            for filename in file_list:
                print(file_path, "(", count, "/", total,"): ", filename)
                with open(os.path.join(file_path, filename)) as f:
                    while f.readline():
                        result = f.readline()
                        if result[:2] == '..':
                            info = result
                            result = 'none'
                        else:
                            if result[:2] == './' or result[:2] == 'cv' or result[:2] == 'ca':
                                result = 'timeout'
                            elif result[:2] == 'tc':
                                result = 'leak'
                            info = f.readline()
                            if info[:2] == '(e':
                                break
                            while info[:2] == './' or info[:2] == 'tc' or info[:2] == 'cv' or info[:2] == 'ca':
                                info = f.readline()
                            if info[:2] != '..':
                                result = info
                                info = f.readline()
                        info = info.split(' : ')
                        records.loc[records['filename'] == info[0].split('//')[1], [result_index, timeout_index]] = [result, info[1][:-2]]
                count += 1
 

In [ ]:
path = "Data_File"
folder_list = os.listdir(path)
log = open('log.txt', 'w')

for filefolder in folder_list:
    if filefolder == 'smtrat':
        folder_path = path + "/" + filefolder
        times_list = os.listdir(folder_path)
        for filetimes in times_list:
            file_path = folder_path + "/" + filetimes
            file_list = os.listdir(file_path)
            result_index = filefolder + "_" + filetimes + "_Result"
            timeout_index = filefolder + "_" + filetimes + "_Timeout"
            total = len(file_list)
            count = 1
            for filename in file_list:
                print(file_path, "(", count, "/", total,"): ", filename)
                with open(os.path.join(file_path, filename)) as f:
                    while f.readline():
                        result = f.readline()
                        if result[:2] == '..':
                            info = result
                            result = 'none'
                        elif result[:2] == 'ER':
                            log.write(file_path + '/' + filename + result + '\n')
                            info = f.readline()
                            while info[:2] != '..':
                                info = f.readline()
                            result = 'invalid operator \"div\"'
                        else:
                            if result[:2] == './' or result[:2] == 'cv' or result[:2] == 'ca':
                                result = 'timeout'
                            elif result[:2] == 'tc':
                                result = 'leak'
                            info = f.readline()
                            if info[:2] == '(e':
                                break
                            while info[:2] == './' or info[:2] == 'tc' or info[:2] == 'cv' or info[:2] == 'ca':
                                info = f.readline()
                            if info[:2] != '..':
                                result = info
                                info = f.readline()
                        info = info.split(' : ')
                        records.loc[records['filename'] == info[0].split('//')[1], [result_index, timeout_index]] = [result, info[1][:-2]]
                count += 1
f.close()

In [ ]:
print("Waiting...")
records.to_excel('Result/records1.xlsx')
print("Done")

保存ismt的结果和时间

In [ ]:
records1 = pd.read_excel(r"Result/records1.xlsx", header=[0], index_col = [0])
records2 = pd.read_excel(r"Result/records1.xlsx", header=[0], index_col = [0])
path = "Data_File"
folder_list = os.listdir(path)
for filefolder in folder_list:
    if filefolder == 'ismt':
        folder_path = path + "/" + filefolder
        times_list = os.listdir(folder_path)
        for filetimes in times_list:
            file_path = folder_path + "/" + filetimes
            file_list = os.listdir(file_path)
            result_index = filefolder + "_" + filetimes + "_Result"
            timeout_index = filefolder + "_" + filetimes + "_Timeout"
            total = len(file_list)
            count = 1
            for filename in file_list:
                print(file_path, "(", count, "/", total,"): ", filename)
                with open(os.path.join(file_path, filename)) as f:
                    while f.readline():
                        result = f.readline()
                        if result[:2] == '..':
                            info = result
                            result = 'none'
                        else:
                            info = f.readline()
                            while info[:2] != '..':
                                result = info
                                info = f.readline()
                            if result[:2] == 're':
                                result = 'sat'
                            elif result[:2] == 'Nu':
                                result = 'timeout'
                        info = info.split(' : ')
                        records1.loc[records1['filename'] == info[0].split('//')[1], [result_index, timeout_index]] = [result, info[1][:-2]]
                        records2.loc[records2['filename'] == info[0].split('//')[1], ['Problem(file name)', result_index, timeout_index, 'group']] = [info[0], result, info[1][:-2], 1]
                count += 1

In [ ]:
records1 = records1.drop(columns = ['filename'])
records2 = records2.drop(columns = ['filename'])
records2 = records2[records2['group'] == 1]
records2 = records2.drop(columns = ['group'])
print("Waiting...")
records1.to_excel('Result/result1.xlsx')
records2.to_excel('Result/result2.xlsx')
print("Done")